# Assignment 5: Clustering
-----
Imagine that you are working as a data scientist in an asset management firm. The firm comes up with an idea to invest in the US stock market. However, the firm does not have much information about the US stock market, especially the diversity of the different stocks. So they come to you and ask for your help. 

## Your Tasks

1. Create an IPython for this assignment and name it assn5.ipynb
2. Extract the S&P 100 stock names from this site: https://en.wikipedia.org/wiki/S%26P_100 (Links to an external site.)
3. For each stock, download the historical price from: http://www.google.com/finance/historical?q=AAPL&output=csv (Links to an external site.)  Replace AAPL with the name of each stock.
4. After you have obtained the prices, normalize the price of each stock to be within [0,1]. We did this as we don't really care about the actual prices but more on the movement of the prices.
5. Define a distance function between two stocks: d(s1,s2). You can first try using the sum of square errors.
6. Perform clustering on the data and answer the following questions:
    * 6.1 How many clusters do you observe?
    * 6.2 What are the stocks within each cluster?
    * 6.3 Do you observe any relationship between your cluster structure and their business sectors?
7. (Optional) Create a visualization of the cluster structure as a 2D scatter plot. Hint: Look into a technique called multidimensional scaling (MDS)

In [1]:
%matplotlib inline
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pylab import *
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import itertools
import matplotlib
import math
from scipy.stats import t
import glob as glob
from pandas import DataFrame, read_csv


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# Create a variable with the URL to this tutorial
url = 'https://en.wikipedia.org/wiki/S%26P_100'

# Scrape the HTML at the url
r = requests.get(url)

# Turn the HTML into a Beautiful Soup object
soup = BeautifulSoup(r.text, "lxml")

In [3]:
soup

<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="en">\n<head>\n<meta charset="unicode-escape"/>\n<title>S&amp;P 100 - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"S\\u0026P_100","wgTitle":"S\\u0026P 100","wgCurRevisionId":768116882,"wgRevisionId":768116882,"wgArticleId":2658424,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["American stock market indices","S\\u0026P Dow Jones Indices"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","J

In [4]:

# print soup.find('table',{'class': 'wikitable sortable jquery-tablesorter'})
# ---------
# for table in soup.find_all('table'):
#     for tr in table.find_all('tr'):
#         for td in tr.find_all('td'):
#             print td
# ---------
all_stock_name = []

for all_td in soup.find_all('td'):
    td = all_td.get_text()
    if len(td) <= 6 and td.isalpha() and len(str(all_td))<= 15 :
        all_stock_name.append(td)
print len(all_stock_name), all_stock_name        
    


101 [u'AAPL', u'ABBV', u'ABT', u'ACN', u'AGN', u'AIG', u'ALL', u'AMGN', u'AMZN', u'AXP', u'BA', u'BAC', u'BIIB', u'BK', u'BLK', u'BMY', u'C', u'CAT', u'CELG', u'CL', u'CMCSA', u'COF', u'COP', u'COST', u'CSCO', u'CVS', u'CVX', u'DD', u'DHR', u'DIS', u'DOW', u'DUK', u'EMR', u'EXC', u'F', u'FB', u'FDX', u'FOX', u'FOXA', u'GD', u'GE', u'GILD', u'GM', u'GOOG', u'GOOGL', u'GS', u'HAL', u'HD', u'HON', u'IBM', u'INTC', u'JNJ', u'JPM', u'KHC', u'KMI', u'KO', u'LLY', u'LMT', u'LOW', u'MA', u'MCD', u'MDLZ', u'MDT', u'MET', u'MMM', u'MO', u'MON', u'MRK', u'MS', u'MSFT', u'NEE', u'NKE', u'ORCL', u'OXY', u'PCLN', u'PEP', u'PFE', u'PG', u'PM', u'PYPL', u'QCOM', u'RTN', u'SBUX', u'SLB', u'SO', u'SPG', u'T', u'TGT', u'TWX', u'TXN', u'UNH', u'UNP', u'UPS', u'USB', u'UTX', u'V', u'VZ', u'WBA', u'WFC', u'WMT', u'XOM']


In [108]:
Location = 'aapl.csv'
df = pd.read_csv(Location)

In [109]:
df

,Date,Open,High,Low,Close,Volume
0,7-Apr-17,143.73,144.18,143.27,143.34,16672198
1,6-Apr-17,144.29,144.52,143.45,143.66,21149034
2,5-Apr-17,144.22,145.46,143.81,144.02,27717854
3,4-Apr-17,143.25,144.89,143.17,144.77,19891354
4,3-Apr-17,143.71,144.12,143.05,143.70,19985714
5,31-Mar-17,143.72,144.27,143.01,143.66,19661651
6,30-Mar-17,144.19,144.50,143.50,143.93,21207252
7,29-Mar-17,143.68,144.49,143.19,144.12,29189955
8,28-Mar-17,140.91,144.04,140.62,143.80,33374805
9,27-Mar-17,139.39,141.22,138.62,140.88,23575094
